In [1]:
import pandas as pd
import numpy as np

In [2]:
def estimateparamsbayes(X,questions,allbest,allworst,w0=[],weights=[],tol=.0001,maxiter=100):
    tol=.001
    val=1
    [numofq,nitems]=X.shape
    if len(w0)<nitems or (np.abs(w0)>4).any():
        w=np.zeros(nitems)    
    else:
        w=w0.copy()
    if len(weights)<numofq:
        weights=np.ones(numofq)
    iternum=0
    nb=np.bincount(allbest,weights,minlength=nitems)
    nw=np.bincount(allworst,weights,minlength=nitems)
    try:
        while val>tol and iternum<maxiter:
            zb=np.sum(np.exp(w[questions]),1)
            zw=np.sum(np.exp(-w[questions]),1)
            Pb=((1/zb*X.T).T*np.exp(w))#broadcasting
            Pw=((1/zw*X.T).T*np.exp(-w))#broadcasting
            wPb=(Pb.T*weights).T
            wPw=(Pw.T*weights).T
            gradb=nb-np.sum(wPb,0)
            gradw=-nw+np.sum(wPw,0)
            grad=-(gradb+gradw)
            Hb=np.dot(wPb.T,Pb)
            Hbd=-np.sum(wPb*(1-Pb),0)
            Hb+=-np.diag(np.diag(Hb))+np.diag(Hbd)
            Hw=np.dot(wPw.T,Pw)
            Hwd=-np.sum(wPw*(1-Pw),0)
            Hw+=-np.diag(np.diag(Hw))+np.diag(Hwd)
            H=-(Hb+Hw)
            gradl=grad[:-1]
            Hl=H[:-1,:-1]
            Hinvgradl=np.linalg.solve(Hl,gradl)
            w[:-1]=w[:-1]-Hinvgradl
            val=np.dot(gradl,Hinvgradl)
            iternum+=1
    except np.linalg.linalg.LinAlgError:
        return np.random.randn(nitems)/4
    if np.isnan(w).any():
        raise Exception('w contains nan')
    return w

In [ ]:
def calculatePVR(filename="TS-120v20"):
    data=pd.read_csv('Results/choice_data-'+filename+'.csv')
    bhist=pd.read_csv('Results/bhist-'+filename+'.csv')
    itemcol=[]
    itemnum=120
    for i in range(1,itemnum+1):
        itemcol+=['Item_'+str(i)]
    iters=int(np.max(bhist['Iter']))
    respnums=(bhist[bhist['Iter']==1]['RespNum'].get_values()).astype(int)
    resplen=len(respnums)
    choicecol=['Choice1','Choice2', 'Choice3', 'Choice4', 'Choice5']
    pvrfull=np.zeros((resplen*iters,3))
    for iternum in range(1,iters+1):
        for i,resp in enumerate(respnums):
            choices=data[np.logical_and(data['Iter']==iternum,data['RespIDNum']<=resp)][choicecol].get_values()-1
            best=data[np.logical_and(data['Iter']==iternum,data['RespIDNum']<=resp)]['Best'].get_values()-1
            worst=data[np.logical_and(data['Iter']==iternum,data['RespIDNum']<=resp)]['Worst'].get_values()-1
            curbeta=bhist[np.logical_and(bhist['Iter']==iternum,bhist['RespNum']==resp)][itemcol].get_values()[0]
            [numofq,items]=choices.shape
            X=np.zeros((numofq,itemnum))
            for z,row in enumerate(choices):
                X[z,row]=1
            N=100
            draws=np.zeros((N,itemnum))
            for j in range(N):
                weights=np.random.exponential(1,numofq)
                draws[j]=estimateparamsbayes(X,choices,best,worst,weights=weights)
            k=10
            rank=np.argsort(curbeta)[::-1]
            valstar=np.sum(np.exp(draws[:,rank[:k]]),1)
            valmax=np.sum(np.exp(np.sort(draws,1)[:,::-1][:,:k]),1)
            vrhist=(valmax-valstar)/valstar
            pvrfull[(iternum-1)*resplen+i,0]=iternum
            pvrfull[(iternum-1)*resplen+i,1]=resp
            pvrfull[(iternum-1)*resplen+i,2]=np.percentile(vrhist,95)
    pvrfull_df=pd.DataFrame(pvrfull)
    pvrfull_df.columns=["Iter","RespNum","PVR"]
    pvrfull_df.to_csv("Results/PVR-"+filename+".csv")

In [27]:
pvrfull_df.to_csv("Results/PVR-"+filename+".csv")

6.666666666666667

In [28]:
add=''
pvrfull_df.to_csv("Results/PVR-"+filename+add+".csv")

4.0